# Overview

## Description

**Why This Matters**

Accurate sales forecasts are crucial for planning process, supply chain processes, delivery logistics and inventory management. By optimizing forecasts, we can minimize waste and streamline operations, making our e-grocery services more sustainable and efficient.

**Your Impact**

Your participation in this challenge will directly contribute to Rohlik mission of sustainable and efficient e-grocery delivery. Your insights will help us enhance customer service and achieve a greener future.

We are relaunching the Challenge with prizes.

Rohlik Group, a leading European e-grocery innovator, is revolutionising the food retail industry. We operate across 11 warehouses in Czech Republic, Germany, Austria, Hungary, and Romania.

We are now transitioning from the Rohlik Orders Forecasting Challenge to the Rohlik Sales Forecasting Challenge, as we continue with our set of challenges. This challenge focuses on predicting the sales of each selected warehouse inventory for next 14 days using historical sales data.

## Evaluation

Submissions are evaluated on Weighted Mean Absolute Error (WMAE) between the predicted sales and the actual sales. Weights for the test evaluation can be found in the Data section.

## Submission File

For each ID in the test set, you must predict a probability for the TARGET variable. The file should contain a header and have the following format:

id,sales_hat

840_2024-06-10,12.01

2317_2024-06-15,13.32

738_2024-06-10,14.12

3894_2024-06-11,3.03

3393_2024-06-08,53.03


## Prizes
Leaderboard prizes

1st place - $4,000

2nd place - $4,000

3rd place - $2,000

## Citation

MichalKecera. Rohlik Sales Forecasting Challenge. https://kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2, 2024. Kaggle.

# Data

## Dataset Description
You are provided with historical sales data for selected Rohlik inventory and date. IDs, sales, total orders and price columns are altered to keep the real values confidential. Some features are not available in test as they are not known at the moment of making the prediction. The task is to forecast the sales column for a given id, constructed from unique_id and date (e. g. id 1226_2024-06-03 from unique_id 1226 and date 2024-06-03), for the test set.



## Files
- **sales_train.csv** - training set containing the historical sales data for given date and inventory with selected features described below
- **sales_test.csv** - full testing set
- **inventory.csv** - additional information about inventory like its product (same products across all warehouses share same product unique id and name, but have different unique id)
- **solution.csv** - full submission file in the correct format
- **calendar.csv** - calendar containing data about holidays or warehouse specific events, some columns are already in the train data but there are additional rows in this file for dates where some warehouses could be closed due to public holiday or Sunday (and therefore they are not in the train set)



## Columns
**sales_train.csv** and **sales_test.csv**

- `unique_id` - unique id for inventory
- `date` - date
- `warehouse` - warehouse name
- `total_orders` - historical orders for selected Rohlik warehouse known also for test set as a part of this challenge
- `sales` - Target value, sales volume (either in pcs or kg) adjusted by availability. The sales with lower availability than 1 are already adjusted to full potential sales by Rohlik internal logic. There might be missing dates both in train and test for a given inventory due to various reasons. This column is missing in test.csv as it is target variable.
- `sell_price_main` - sell price
- `availability` - proportion of the day that the inventory was available to customers. The inventory doesn't need to be available at all times. A value of 1 means it was available for the entire day. This column is missing in test.csv as it is not known at the moment of making the prediction.
- `type_0_discount`, type_1_discount, … - Rohlik is running different types of promo sale actions, these show the percentage of the original price during promo ((original price - current_price) / original_price). Multiple discounts with different type can be run at the same time, but always the highest possible discount among these types is used for sales. Negative discount value should be interpreted as no discount.

**inventory.csv**

- `unique_id` - inventory id for a single keeping unit
- `product_unique_id` - product id, inventory in each warehouse has the same product unique id (same products across all warehouses has the same product id, but different unique id)
- `name` - inventory id for a single keeping unit
L1_category_name, L2_category_name, … - name of the internal category, the higher the number, the more granular information is present
- `warehouse` - warehouse name

**calendar.csv**

- `warehouse` - warehouse name
- `date` - date
- `holiday_name` - name of public holiday if any
- `holiday` - 0/1 indicating the presence of holidays
- `shops_closed` - public holiday with most of the shops or large part of shops closed
- `winter_school_holidays` - winter school holidays
- `school_holidays` - school holidays

**test_weights.csv**

- `unique_id` - inventory id for a single keeping unit
- `weight` - weight used for final metric computation

# EDA


In [108]:
# import
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [109]:
sales_train = pd.read_csv('sales_train.csv')
sales_test = pd.read_csv('sales_test.csv')
weights = pd.read_csv('test_weights.csv')
calendar = pd.read_csv('calendar.csv')
inventory = pd.read_csv('inventory.csv')

In [110]:
sales_train.head()


,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0


In [111]:
sales_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   unique_id        int64  
 1   date             object 
 2   warehouse        object 
 3   total_orders     float64
 4   sales            float64
 5   sell_price_main  float64
 6   availability     float64
 7   type_0_discount  float64
 8   type_1_discount  float64
 9   type_2_discount  float64
 10  type_3_discount  float64
 11  type_4_discount  float64
 12  type_5_discount  float64
 13  type_6_discount  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 428.0+ MB


In [112]:
sales_train.isna().sum()


unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64

In [113]:
#dropping missing values
sales_train.dropna(inplace=True)


In [114]:
sales_train.nunique()


unique_id            5390
date                 1402
warehouse               7
total_orders         7508
sales              127279
sell_price_main     35723
availability          100
type_0_discount     17786
type_1_discount       154
type_2_discount      3521
type_3_discount        18
type_4_discount      1019
type_5_discount       675
type_6_discount     44658
dtype: int64

In [115]:
sales_test.head()


,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
sales_test.isna().sum()


unique_id          0
date               0
warehouse          0
total_orders       0
sell_price_main    0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_discount    0
type_4_discount    0
type_5_discount    0
type_6_discount    0
dtype: int64

In [117]:
sales_train.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0


In [118]:
# merge sales tables with calendar and inventory 
sales_train = pd.merge(sales_train, calendar, how = 'inner', on = ['date','warehouse'])
sales_train = pd.merge(sales_train,inventory, how = 'inner', on = ['unique_id','warehouse'])
sales_test = pd.merge(sales_test, calendar, how = 'inner', on = ['date','warehouse'])
sales_test = pd.merge(sales_test,inventory, how = 'inner', on = ['unique_id','warehouse'])

# Feature Engineering

In [94]:
# #create column for country based on warehouse 
# warehouse_to_country = {'Prague':'Czech Republic','Brno':'Czech Republic','Budapest':'Hungary','Munich':'Germany','Frankfurt':'Germany'}
# sales_train['country']=sales_train['warehouse'].str.split('_').str[0].map(warehouse_to_country)

# # create discount column
# sales_train['discount']=sales_train[['type_0_discount','type_1_discount','type_2_discount','type_3_discount','type_4_discount','type_5_discount','type_6_discount']].max(axis = 1)

# #calculate avg sales per order
# sales_train['sales_per_order'] = sales_train['sales']/sales_train['total_orders']

# # drop availability column
# sales_train = sales_train.drop('availability', axis=1)

In [120]:
for i, df in enumerate([sales_train, sales_test]):
    #create column for country based on warehouse 
    warehouse_to_country = {'Prague':'Czech Republic','Brno':'Czech Republic','Budapest':'Hungary','Munich':'Germany','Frankfurt':'Germany'}
    df['country']=df['warehouse'].str.split('_').str[0].map(warehouse_to_country)

    # create discount column
    df['discount'] = df.loc[:, 'type_0_discount':'type_6_discount'].max(axis=1)
   
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])  
    
    # Create columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.day_of_week
    
    # One-hot encoding
    df = pd.get_dummies(df, columns=['warehouse'], dtype=int)
       
    # Dropping unnecessary columns
    df.drop(columns=[
        'type_0_discount', 'type_1_discount', 'type_2_discount', 
        'type_3_discount', 'type_4_discount', 'type_5_discount', 
        'type_6_discount'
    ], inplace=True)
    
    # Assign back to the original DataFrame
    if i == 0:
        sales_train = df
    else:
        sales_test = df
        
# drop availability column
sales_train = sales_train.drop('availability', axis=1)

In [96]:
# Create bins for the 'sales' column
sales_train['sales_bins'] = pd.qcut(sales_train['sales'], q=4, labels=False)  # Quartiles (4 bins)

In [97]:
sales_train.head()

,unique_id,date,total_orders,sales,sell_price_main,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,...,day,weekday,warehouse_Brno_1,warehouse_Budapest_1,warehouse_Frankfurt_1,warehouse_Munich_1,warehouse_Prague_1,warehouse_Prague_2,warehouse_Prague_3,sales_bins
0,4845,2024-03-10,6436.0,16.34,646.26,NaN,0,0,0,0,...,10,6,0,1,0,0,0,0,0,0
1,4845,2021-05-25,4663.0,12.63,455.96,NaN,0,0,0,0,...,25,1,0,1,0,0,0,0,0,0
2,4845,2021-12-20,6507.0,34.55,455.96,NaN,0,0,0,0,...,20,0,0,1,0,0,0,0,0,1
3,4845,2023-04-29,5463.0,34.52,646.26,NaN,0,0,0,0,...,29,5,0,1,0,0,0,0,0,1
4,4845,2022-04-01,5997.0,35.92,486.41,NaN,0,0,0,0,...,1,4,0,1,0,0,0,0,0,1


In [98]:
sales_train.isna().sum()

unique_id                       0
date                            0
total_orders                    0
sales                           0
sell_price_main                 0
holiday_name              3844077
holiday                         0
shops_closed                    0
winter_school_holidays          0
school_holidays                 0
product_unique_id               0
name                            0
L1_category_name_en             0
L2_category_name_en             0
L3_category_name_en             0
L4_category_name_en             0
country                         0
discount                        0
year                            0
month                           0
day                             0
weekday                         0
warehouse_Brno_1                0
warehouse_Budapest_1            0
warehouse_Frankfurt_1           0
warehouse_Munich_1              0
warehouse_Prague_1              0
warehouse_Prague_2              0
warehouse_Prague_3              0
sales_bins    

In [99]:
# # considering the number of nan values, I am dropping holiday_name momentarily
# sales_train.drop(columns='holiday_name', axis=1, inplace=True)
# sales_test.drop(columns='holiday_name', axis=1,inplace=True)

# Prediction Model


## Preprocessing


In [122]:
# isolating all numeric data
sales_train = sales_train.select_dtypes('number')
sales_test = sales_test.select_dtypes('number')

In [101]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(sales_train,
                            test_size=0.3, 
                            stratify=sales_train['sales_bins'], 
                            random_state=1910)

In [102]:
# Drop the 'sales_bins' column from train and test sets
train = train.drop(columns=['sales_bins'])
val = val.drop(columns=['sales_bins'])
X_train = train.drop(columns=['sales'])
y_train = train['sales']
X_val = val.drop(columns=['sales'])
y_val = val['sales']

## Regressor Model


In [103]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(oob_score=True)

reg.fit(X_train, y_train)


RandomForestRegressor(oob_score=True)

In [104]:
from sklearn.metrics import mean_absolute_error

predictions = reg.predict(X_val)

mae = mean_absolute_error(y_val,predictions)

print("MAE:", mae)


In [105]:
# Compute WMAE
def calculate_wmae(actual, predicted, weights):
    return np.sum(weights * np.abs(actual - predicted)) / np.sum(weights)
calculate_wmae(y_val, predictions, weights['weight'])


np.float64(3.778017695620432)

# Solution

In [125]:
solution = pd.read_csv('solution.csv')
solution['sales_hat'] = reg.predict(sales_test)



In [126]:
#Export final solution
solution.to_csv('solution.csv', index=False)